# 01 - First Exploration

## System 

In [ ]:
cd ../

In [ ]:
pwd

## Imports

In [ ]:
import os, sys, logging

from dataclasses import dataclass

In [ ]:
from IPython.display import display, HTML

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import missingno as msno

In [ ]:
from gbs.etl.extract import Extract

## Data

In [ ]:
extract = Extract()
extract

In [ ]:
data_dir = os.path.join(
    # os.getcwd(),
    extract.base,
    extract.folder,
    extract.subfolder,
)
data_dir

In [ ]:
# # get data
# if (not os.path.exists(data_dir)) or  (not os.listdir(data_dir)):
#     extract.get_all(clean=True, include_production=True)

In [ ]:
extract.get_all(clean=True, include_production=True)

In [ ]:
data_dir = "./data/source/"
os.listdir(data_dir)

In [ ]:
!rm data/source/production.zip

In [ ]:
!tree -L 1 ./

In [ ]:
!tree -L 3 data/

In [ ]:
crops = pd.read_csv(os.path.join(data_dir, "crops.csv"))
crops

In [ ]:
country_specs = pd.read_csv(os.path.join(data_dir, "country_specs.csv"))
country_specs.head()

In [ ]:
_path = "./data/source/production/"

fn_list = [
    os.path.join(_path, f) for f in os.listdir(_path) if f.endswith(".csv")
]
fn_list

In [ ]:
for fn in fn_list:
    # read file
    with open(fn, "r", encoding="latin-1") as f:
        txt = f.read()

    # clean file
    txt.replace(",'", ",")

    # write file
    with open(fn, "w", encoding="utf8") as f:
        f.write(txt)

In [ ]:
_path = "./data/source/production/"

In [ ]:
@dataclass
class Production:
    """Production data for a crop in a country in a year"""

    areacodes = pd.read_csv(os.path.join(_path, "areacodes.csv"))
    data_normalized = pd.read_csv(os.path.join(_path, "data_normalized.csv"))
    flags = pd.read_csv(os.path.join(_path, "flags.csv"))
    itemcodes = pd.read_csv(os.path.join(_path, "itemcodes.csv"))

## First exploration 

### Crops 

#### Display

In [ ]:
crops.head(10)

In [ ]:
crops.tail(10)

In [ ]:
crops.sample(10)

#### Structure

In [ ]:
crops.shape

In [ ]:
crops.columns

In [ ]:
crops.dtypes

In [ ]:
crops.info()

In [ ]:
crops.dtypes.value_counts()

In [ ]:
for dtype in ["object", "float", "int"]:
    selected_dtype = crops.select_dtypes(include=[dtype])
    display(selected_dtype.columns)

In [ ]:
_num = crops.select_dtypes(include=["number"])
_num

In [ ]:
_num_cols = _num.columns.tolist()
[i for i in _num_cols if ("code" not in i) and ("id" not in i)]

**Conclusion**

No data in the crop dataset.

#### Nan

In [ ]:
crops.isna().sum()

In [ ]:
tmp = crops.isna().mean().round(2)
tmp[tmp > 0.00]

In [ ]:
len(tmp[tmp > 0.00]) / len(tmp)

In [ ]:
tmp = crops.isna().mean(axis=1).round(2)
tmp.value_counts().sort_index()

In [ ]:
msno.matrix(crops)

**Conclusion**

- delete is_EU27 and is_south_america

In [ ]:
# delete is_EU27 and is_south_america

In [ ]:
drop_cols = ["is_EU27", "is_south_america"]
_crops = crops.drop(columns=drop_cols)
msno.matrix(_crops)

In [ ]:
tmp = _crops.isna().mean(axis=1).round(2)
tmp.value_counts().sort_index()

In [ ]:
tmp[tmp > 0.33]

In [ ]:
drop_idxs = tmp[tmp > 0.33].index
crops.loc[drop_idxs, :]

**Conclusion**

- countries with Nan rate > 0.3 are Island or -100k pop (Monaco, Antigua)

In [ ]:
_crops = crops.drop(index=drop_idxs, columns=drop_cols)
msno.matrix(_crops)

In [ ]:
_crops

In [ ]:
num_cols = _crops.select_dtypes(include=["float", "int"]).columns.tolist()
num_cols = [i for i in num_cols if "code" not in i]
categ_cols = ["alpha_3_code", "FAO_country_name", "exiobase_region_name"]


_crops.loc[:, num_cols + categ_cols]

**Conclusion**

- No relevant data in the crop dataset

#### DataSet Conclusion

- No relevant data in the crop dataset